<a href="https://colab.research.google.com/github/Ash111998/fromCMD/blob/main/ISYE_6202_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shared Storage Shipment to Flow Assignment Optimisation based on Maximum Flow

In [ ]:
!pip install ortools

In [ ]:
import pandas as pd
import math
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy
import scipy.stats
import xlrd
from ortools.linear_solver import pywraplp


workbook = xlrd.open_workbook("/Python Input Data 2.xlsx")
sheet = workbook.sheet_by_name("Sheet1")
data = [[sheet.cell_value(row, col) for col in range(sheet.ncols)] for row in range(sheet.nrows)]

print(data)

In [ ]:
def assign(flow):
    num_shipment = len(flow)
    num_zone = len(flow[0])

    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if  Shipment i is assigned Zone j.
    x = {}
    for i in range(num_shipment):
        for j in range(num_zone):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each Shipment is assigned to maximum 3 Zones.
    for i in range(num_shipment):
        solver.Add(solver.Sum([x[i, j] for j in range(num_zone)]) <= 3)

    # Each Zone should be assigned to atleast 2 Shipments.
    for j in range(num_zone):
        solver.Add(solver.Sum([x[i, j] for i in range(num_shipment)]) >= 2)

    # Shipment 4 and 5 should be in the same zone.
    for j in range(num_zone):
        solver.Add(x[3,j] == x[4,j])

    #Each Zone should have a mix of High and Low priority shipments with atleast 1 High priority shipments
    for j in range(num_zone):
        solver.Add(solver.Sum([x[i, j] for i in range(5)]) >= 1)

    #Shipments with  'High' Priority should be assigned to zones closest to Shipping and Receiving (Zone 6,8,10,12)
    for j in range(3,8):
        solver.Add(solver.Sum([x[0,j], x[1,j], x[2,j], x[3,j], x[4,j]]) >= 1)

    # Objective
    objective_terms = []
    for i in range(num_shipment):
        for j in range(num_zone):
            objective_terms.append(flow[i][j] * x[i, j])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Objective Value = ', solver.Objective().Value(), '\n')
        for i in range(num_shipment):
            for j in range(num_zone):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    #print('Student %d assigned to Class %d',.)
                    print("%d,%d" %(i+1,j+1))

In [ ]:
assignedSol = assign(data)

# Dead Head Reduction for Dedicated Layout

In [ ]:
!pip install ortools

In [ ]:
import pandas as pd
import math
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy
import scipy.stats
import xlrd
from ortools.linear_solver import pywraplp


workbook = xlrd.open_workbook("/Python Input Data.xlsx")
sheet = workbook.sheet_by_name("Sheet1")
data = [[sheet.cell_value(row, col) for col in range(sheet.ncols)] for row in range(sheet.nrows)]

print(data)

In [ ]:
def assign(distance):
    num_stow = len(distance)
    num_retrieve = len(distance[0])

    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if Stow-at-Zone i is assigned Retrieval-at-Zone j.
    x = {}
    for i in range(num_stow):
        for j in range(num_retrieve):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each Stow is assigned to exactly 1 Retrieval.
    for i in range(num_stow):
        solver.Add(solver.Sum([x[i, j] for j in range(num_retrieve)]) == 1)

    # Each Retrieval is assigned to exactly 1 Stow.
    for j in range(num_retrieve):
        solver.Add(solver.Sum([x[i, j] for i in range(num_stow)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_stow):
        for j in range(num_retrieve):
            objective_terms.append(distance[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total distance = ', solver.Objective().Value(), '\n')
        for i in range(num_stow):
            for j in range(num_retrieve):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    #print('Stow %d assigned to Retrieve %d.  Distance = %d' %(i, j, distance[i][j]))
                    print("%d,%d" %(i,j))

In [ ]:
assignedSol = assign(data)

# Analysis of Dedicated and Class Based Layout based on Static Demand

The Source Code of the Python was divided into 6 functions which is as follows: 

1. Class wise Bang for Buck SKU Assignment
2. Bang for Buck Optimisation
3. Layout design and flow matrix for Dedicated (Same R&S)
4. Layout design and flow matrix for Class (SameR&S)
5. Layout Design and Flow Matrix for Dedicated (Opposite R&S)
6. Layout Design and Flow Matrix for Class (Opposite R&S)
7. Layout Design and Flow Matrix for Shared (Same R & S)
8. Layout Design and Flow Matrix for Shared (Different R & S)


The Bang for Buck Class Assignment takes the value of SKU, Carton Demand, Pallets Required (Max bays), Slots and Benefits and provides up with the Flow and Total Distance Travelled Matrix Along with Feets / move.


**Function 1: Class Assignment for Bang for Buck *Optimisation***

In [1]:
def classs(c):
    import pandas as pd
    sku=[]
    carton=[]
    pr=[]
    mpp=0
    br=0
    b=[]
    r=[]
    s=0
    sa=[]
    sp=0
    cpp=[]
    sl=[]
    z=[]
    rf=[]
    average=0
    df=pd.read_excel(c,sheet_name='Class Flow')
    n=len(df)
    for i in range (0,n):
        sku.append(df.iloc[i,0])
        carton.append(df.iloc[i,1])
        pr.append(df.iloc[i,2])
        sl.append(df.iloc[i,3])
        b.append(df.iloc[i,4])
        mpp=carton[i]+mpp
        br=br+carton[i]
    average=mpp/br
    for i in range (0,n):
        r.append(int(carton[i])/int(pr[i]))
        rf.append(int(carton[i])/(sl[i]))
    df["Bays Required"]=r
    df["Flow Based on Slots"]=rf
    df["Benefits"]=b
    df=df.sort_values(by='Flow Based on Slots', ascending=False, na_position='first')
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    o=int(input("Enter the Number of SKU's to be assiged for Class A"))
    for i in range (0,n):
        if (i<=o):
            z.append("Class A")
        else:
            z.append("Class B")
    df["Class"]=z
    df.to_excel(r'Export SKU Assignment Class Wise',sheet_name='Flow',index=False)

## **Function 2: Bang for Buck *Optimisation***



The Function 2 carries out the Bang for Buck Optimisation which provides the All Space, Some Space, More Space requirements for each SKU over a particular time period (Static Demand)

In [2]:
def bang(c):
    import pandas as pd
    sku=[]
    carton=[]
    ppp=[]
    cdi=[]
    fpd=[]
    mnl=[]
    al=[]
    sl=[]
    ml=[]
    j=0
    l=0
    z=[]
    df=pd.read_excel(c,sheet_name='Class Based Split')
    mp=[200,120,80,70,90,80,75,20,5,17,13,5,11,5]
    n=len(df)
    s=int(input("Enter the Forward Pick Savings"))
    r=int(input("Enter the Restocking Stock"))
    for i in range (0,n):
        sku.append(df.iloc[i,0])
        carton.append(df.iloc[i,1])
        ppp.append(df.iloc[i,2])
        cdi.append(int(ppp[i])/2)
        fpd.append(int(carton[i])/ppp[i])
        mnl.append(int(carton[i])/50)
        al.append(((s)*(carton[i]+fpd[i]))/mp[i])
        j=(float(((s)*(carton[i]))-(r*cdi[i])))
        sl.append(j/mnl[i])
        l=(float(((s)*(fpd[i])+(r*cdi[i]))))
        ml.append(l/(mp[i]-mnl[i]))
    df["Carton Demand in Pallets"]=cdi
    df["Full Pallet Demand"]=fpd
    df["Minimum Number of Locations"]=mnl
    df["Maximum Pallets"]=mp
    df["All Space"]=al
    df["Some Space"]=sl
    df["More Space"]=ml
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    df.to_excel(r'Export Bang for Buck Files.xlsx',sheet_name='Flow',index=False)
    print(df)

## **Function 3: Dedicated Layout for Same Receiving and Shipping Side**

The Function 3 provides us the flow and total distance travelled for each SKU over a particular time period (Static Demand). Also, it displays the total distance travelled, feets/move and the Moves/period. 

In [3]:
def layout(c):
    import pandas as pd
    import pandas as pd
    sku=[]
    mpd=[]
    mba=[]
    mba1=[]
    mpp=0
    br=0
    r=[]
    f=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    d=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    dt=0
    a=0
    b=8
    k=0
    y=0
    average=0
    df=pd.read_excel(c,sheet_name='Dedicated')
    n=len(df)
    for i in range(0,n):
        sku.append(df.iloc[i,0])
        mpd.append(df.iloc[i,1])
        mba.append(df.iloc[i,2])
        mpp=mpd[i]+mpp
        br=br+mba[i]
    a=br/80
    print(a)
    for i in range(0,n):
        mba1.append(int(mba[i])/float(a))
        o=b*mba1[i]
        r.append(float(mpd[i])/float(o))
    df["Max Bays Required"]=mba1
    df['Flow']=r
    df=df.sort_values(by='Flow', ascending=False, na_position='first')
    print(df)
    dp=pd.read_excel(c,sheet_name='Distance')
    ds=pd.read_excel(c,sheet_name='SKU Assignment')
    for i in range (0,10):
        for j in range (0,8):
            for k in range (0,n):
                if(ds.iloc[i,j]==df.iloc[k,0]):
                    f[i][j]=(df.iloc[k,4])
                    d[i][j]=f[i][j]*dp.iloc[i,j]
                    dt=dt+d[i][j]
    l=int(input("Enter the total moves/period"))
    y=dt/l
    print("Total Distance Travelled=",dt)
    print("Total Feets/move=",y)
    distance=pd.DataFrame(d)
    flow=pd.DataFrame(f)
    flow.to_excel(r'Export Flow Dedicated.xlsx',sheet_name='Flow',index=False)
    distance.to_excel(r'Export Distance Dedicated.xlsx',sheet_name="Distance",index=False)
    print("The Flow Matrix and Distance Travelled matrix has been generated")

## **Function 4 : Dedicated Layout for Opposite Receiving and Shipping Side**

The Function 4 provides us the flow and total distance travelled for each SKU over a particular time period (Static Demand) for Dedicated Storage. Also, it displays the total distance travelled, feets/move and the Moves/period.

Note: This is for Different Receiving and Shipping Sides

In [4]:
def layoutdedicatedopposite(c):
    import pandas as pd
    import pandas as pd
    sku=[]
    mpd=[]
    mba=[]
    mba1=[]
    mpp=0
    br=0
    r=[]
    f=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    d=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    dt=0
    a=0
    b=8
    k=0
    y=0
    average=0
    df=pd.read_excel(c,sheet_name='Dedicated')
    n=len(df)
    for i in range(0,n):
        sku.append(df.iloc[i,0])
        mpd.append(df.iloc[i,1])
        mba.append(df.iloc[i,2])
        mpp=mpd[i]+mpp
        br=br+mba[i]
    a=br/80
    print(a)
    for i in range(0,n):
        mba1.append(int(mba[i])/float(a))
        o=b*mba1[i]
        r.append(float(mpd[i])/float(o))
    df["Max Bays Required"]=mba1
    df['Flow']=r
    df=df.sort_values(by='Flow', ascending=False, na_position='first')
    print(df)
    dp=pd.read_excel(c,sheet_name='Distance Opposite')
    ds=pd.read_excel(c,sheet_name='SKU Assignment Opposite')
    for i in range (0,10):
        for j in range (0,8):
            for k in range (0,n):
                if(ds.iloc[i,j]==df.iloc[k,0]):
                    f[i][j]=(df.iloc[k,4])
                    d[i][j]=f[i][j]*dp.iloc[i,j]
                    dt=dt+d[i][j]
    l=int(input("Enter the total moves/period"))
    y=dt/l
    print("Total Distance Travelled=",dt)
    print("Total Feets/move=",y)
    distance=pd.DataFrame(d)
    flow=pd.DataFrame(f)
    flow.to_excel(r'Export Flow Dedicated Opposite.xlsx',sheet_name='Flow',index=False)
    distance.to_excel(r'Export Distance Dedicated Opposite.xlsx',sheet_name="Distance",index=False)
    print("The Flow Matrix and Distance Travelled matrix has been generated")

### **Function 5 : Class Based Layout for Same Receiving and Shipping Side**

The Function 5 provides us the flow and total distance travelled for each SKU over a particular time period (Static Demand) for Class Storage. Also, it displays the total distance travelled, feets/move and the Moves/period.

Note: This is for Same Receiving and Shipping Sides

In [5]:
def layoutclass(c):
    import pandas as pd
    import pandas as pd
    sku=[]
    mpd=[]
    mba=[]
    mba1=[]
    mpp=0
    br=0
    r=[]
    f=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    d=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    dt=0
    a=0
    b=8
    k=int(input("Enter total moves/period"))
    average=0
    dp=pd.read_excel(c,sheet_name='Distance')
    ds=pd.read_excel(c,sheet_name='SKU Assignment Class')
    for i in range (0,10):
        for j in range (0,8):
                if(ds.iloc[i,j]=="C1"):
                    f[i][j]=7
                    d[i][j]=f[i][j]*dp.iloc[i,j]
                    dt=dt+d[i][j]
                else:
                    f[i][j]=5
                    d[i][j]=f[i][j]*dp.iloc[i,j]
                    dt=dt+d[i][j]
    a=dt/k
    print("Total Distance Travelled=",dt)
    print("Total Feets/move=",a)
    distance=pd.DataFrame(d)
    flow=pd.DataFrame(f)
    flow.to_excel(r'Export Flow Class.xlsx',sheet_name='Flow',index=False)
    distance.to_excel(r'Export Distance Class.xlsx',sheet_name="Distance",index=False)
    print("The Flow Matrix and Distance Travelled matrix has been generated")


### **Function 6 : Class Based Layout for Opposite Receiving and Shipping Side**

The Function 6 provides us the flow and total distance travelled for each SKU over a particular time period (Static Demand) for Class Storage. Also, it displays the total distance travelled, feets/move and the Moves/period.

Note: This is for Different Receiving and Shipping Sides

In [6]:
def layoutclassopposite(c):
    import pandas as pd
    import pandas as pd
    sku=[]
    mpd=[]
    mba=[]
    mba1=[]
    mpp=0
    br=0
    r=[]
    f=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    d=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    dt=0
    a=0
    b=8
    k=int(input("Enter total moves/period"))
    average=0
    dp=pd.read_excel(c,sheet_name='Distance Opposite')
    ds=pd.read_excel(c,sheet_name='SKU Assignment Class Opposite')
    for i in range (0,10):
        for j in range (0,8):
                if(ds.iloc[i,j]=="C1"):
                    f[i][j]=7
                    d[i][j]=f[i][j]*dp.iloc[i,j]
                    dt=dt+d[i][j]
                else:
                    f[i][j]=5
                    d[i][j]=f[i][j]*dp.iloc[i,j]
                    dt=dt+d[i][j]
    a=dt/k
    print("Total Distance Travelled=",dt)
    print("Total Feets/move=",a)
    distance=pd.DataFrame(d)
    flow=pd.DataFrame(f)
    flow.to_excel(r'Export Flow Class Opposite.xlsx',sheet_name='Flow',index=False)
    distance.to_excel(r'Export Distance Class Opposite.xlsx',sheet_name="Distance",index=False)
    print("The Flow Matrix and Distance Travelled matrix has been generated")

##**Function 7 : Shared Layout for Same Receiving and Shipping Side**

The Function 7 provides us the flow and total distance travelled for each SKU over a particular time period (Static Demand) for Shared Storage. Also, it displays the total distance travelled, feets/move and the Moves/period.

Note: This is for Same Receiving and Shipping Sides



In [7]:
def sharedlayout(c):
    import pandas as pd
    sku=[]
    mpd=[]
    mba=[]
    mba1=[]
    mpp=0
    br=0
    r=[]
    d=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    dt=0
    a=0
    b=int(input("Enter Number of Shipments"))
    k=float(input("Enter Scaling Factor for Distance Calculation"))
    m=int(input("Enter the Moves/period based on the Shipments"))
    average=0
    dp=pd.read_excel(c,sheet_name='Distance')
    ds=pd.read_excel(c,sheet_name='SKU Assignment Shared')
    df=pd.read_excel(c,sheet_name='Shared Flow Same Side')
    for i in range (0,10):
        for j in range (0,8):
                  d[i][j]=df.iloc[i,j]*dp.iloc[i,j]
                  dt=dt+d[i][j]
    a=dt/k
    g=a/b
    print("Total Distance Travelled=",dt)
    print("Scaled Distance",a)
    print("Per Shipment Distance Travelled",g)
    print("Feets/move",g/m)
    distance=pd.DataFrame(d)
    distance.to_excel(r'Export Distance Shared.xlsx',sheet_name="Distance",index=False)
    print("The Flow Matrix and Distance Travelled matrix has been generated")

##**Function 8 : Shared Layout for Different Receiving and Shipping Side**

The Function 8 provides us the flow and total distance travelled for each SKU over a particular time period (Static Demand) for Shared Storage. Also, it displays the total distance travelled, feets/move and the Moves/period.

Note: This is for different Receiving and Shipping Sides



In [8]:
def sharedlayoutopposite(c):
    import pandas as pd
    sku=[]
    mpd=[]
    mba=[]
    mba1=[]
    mpp=0
    br=0
    r=[]
    d=[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
    dt=0
    a=0
    g=0
    b=int(input("Enter Number of Shipments"))
    k=float(input("Enter Scaling Factor for Distance Calculation"))
    m=int(input("Enter the Moves/period based on the Shipments"))
    average=0
    dp=pd.read_excel(c,sheet_name='Distance Opposite')
    ds=pd.read_excel(c,sheet_name='SKU Assignment Opposite Shared')
    df=pd.read_excel(c,sheet_name='Shared Flow Opposite Side')
    for i in range (0,10):
        for j in range (0,8):
                  d[i][j]=df.iloc[i,j]*dp.iloc[i,j]
                  dt=dt+d[i][j]
    a=dt/k
    g=a/b
    print("Total Distance Travelled=",dt)
    print("Scaled Distance",a)
    print("Per Shipment Distance Travelled",g)
    print("Feets/move",g/m)
    distance=pd.DataFrame(d)
    distance.to_excel(r'Export Distance Shared Opposite.xlsx',sheet_name="Distance",index=False)
    print("The Flow Matrix and Distance Travelled matrix has been generated")

## **Function Implementation Code**

In [9]:
i=0
print("Welcome to Warehousing 6202 Project 1")
print("The Files will be imported and exported as excel files in xlsx format")
print("Please choose the Operation to be performed\n1. Class wise Bang for Buck SKU Assignment\n2. Bang for Buck Optimisation\n3. Layout design and flow matrix for Dedicated (Same R&S)\n4. Layout design and flow matrix for Class (SameR&S)\n5. Layout Design and Flow Matrix for Dedicated (Opposite R&S)\n6. Layout Design and Flow Matrix for Class (Opposite R&S)\n7. Layout Design and Flow Matrix for Shared (Same R&S)\n8. Layout Design and Flow Matrix for Shared (Different R & S)")
g=input("Enter the file name of the excel in xlsx format\n")
p=int(input("Enter the function Number\n"))
while(i==0):
    if (p==1):
        classs(g)
    if (p==2):
        bang(g)
    if (p==3):
        layout(g)
    if (p==4):
        layoutclass(g)
    if (p==5):
        layoutdedicatedopposite(g)
    if (p==6):
        layoutclassopposite(g)
    if (p==7):
        sharedlayout(g)
    if (p==8):
        sharedlayoutopposite(g)
    a=int(input(print("Do you wish to terminate or continue another function?\n0 for Yes\n1 for No")))
    if (a==0):
        i=0
        p=int(input("Enter the Function Number"))
    else:
        i==1
        break

Welcome to Warehousing 6202 Project 1
The Files will be imported and exported as excel files in xlsx format
Please choose the Operation to be performed
1. Class wise Bang for Buck SKU Assignment
2. Bang for Buck Optimisation
3. Layout design and flow matrix for Dedicated (Same R&S)
4. Layout design and flow matrix for Class (SameR&S)
5. Layout Design and Flow Matrix for Dedicated (Opposite R&S)
6. Layout Design and Flow Matrix for Class (Opposite R&S)
7. Layout Design and Flow Matrix for Shared (Same R&S)
8. Layout Design and Flow Matrix for Shared (Different R & S)
Enter the file name of the excel in xlsx format
Working File.xlsx
Enter the function Number
4
Enter total moves/period621
Total Distance Travelled= 20152
Total Feets/move= 32.450885668276975
The Flow Matrix and Distance Travelled matrix has been generated
Do you wish to terminate or continue another function?
0 for Yes
1 for No
1
